# Fallback Table Metalearning Prep



In [1]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

In [2]:
# install.packages("pkgcond")

In [4]:
library(dplyr)
library(data.table)
library(partykit)
library(tictoc)
library(caret)
library(e1071)
library(randomForest)
library(ranger)

#for 3d plotting
library(akima)
library(plotly)

# for prep data
library(stringr)
library(pbapply)
library(stringdist)
library(data.table)
library(dominanceanalysis)


getwd()

[1] "/home/jupyter/Domains_202003/scripts/phaseII_06_fallbacksupp"

In [5]:

source('../orig/functions.R')

source('../orig/functions_models.R')

source('../phaseII_03_forest/functions_eval.R')



Attaching package: ‘tidyr’


The following object is masked from ‘package:stringdist’:

    extract




In [6]:
source('functions_metalearning.R')

In [7]:
source('functions_fallback.R')

In [8]:
# source('load_prep_data_expiry_2.R') #read in train test and predictions manually instead

# READ CSV's
# expiry_df_test_preds <- read.csv("../../data/output/datapull_20201116/expiry_df_test_preds.csv")
# expiry_df_train <- read.csv("../../data/output/datapull_20201116/expiry_df_train.csv")
# expiry_df_test <- read.csv("../../data/output/datapull_20201116/expiry_df_test.csv")

# expiry_df
# expiry_train_df 
# expiry_test_df 
# expiry_list
# expiry_train_list
# expiry_test_list

# LOAD fallbacksupp'ed preds

In [9]:
# geo_suppl for train and test-pred data
preds_df <- read.csv("../../data/output/datapull_20201116/expiry_df_test_preds_fb.csv")
head(preds_df)

,reseller,reseller_country,domain_id,renewal_type,renewed_count,expiry_date,creation_date,status,domain,registrar,registrant_country,reg_period,reg_revenue,reg_arpt,renew_mbg,renewal_item_id,renew_type,autorenew_type,renewal_status,renew_date,renew_arpt,gibb_score,pattern,pattern_domain_count,day_domains,⋯,pred_agg_glm,pred_agg_rf,pred_seg_glm_ALL,pred_seg_rf_ALL,pred_seg2_glm_ALL,pred_seg2_rf_ALL,reseller_geo,pred_agg_glm_ALL_fb,pred_agg_rf_ALL_fb,pred_agg_glm_fb,pred_agg_rf_fb,pred_seg_glm_ALL_fb,pred_seg_rf_ALL_fb,pred_seg2_glm_ALL_fb,pred_seg2_rf_ALL_fb,reg_arpt_org,reg_arpt_slab,pred_seg2_rf_ALL_fb2,pred_seg2_glm_ALL_fb2,pred_seg_rf_ALL_fb2,pred_seg_glm_ALL_fb2,pred_agg_rf_fb2,pred_agg_glm_fb2,pred_agg_rf_ALL_fb2,pred_agg_glm_ALL_fb2
,<fct>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<dbl>,<dbl>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,,,122049036,FirstTime,1,2020-08-30,2019-08-30,Active,qianxiyun.online,,NA,1,25,25.00,0,341703158,auto-renewal,unrealized,Not Renewd,2020-08-30,6,0.63,qianxiyun,1,1,⋯,NA,0.7713468,NA,NA,NA,NA,NA,NA,0.9550391,NA,0.7713468,NA,NA,NA,NA,25.00,"(15,25]",0.5510000,0.5510000,0.5510000,0.551,0.7713468,0.551,0.9550391,0.551
2,1&1 Internet,Germany,43450963,FirstTime,1,2020-01-27,2017-01-27,Active,ncyc.online,1&1 Internet,NA,3,55,18.33,0,239731665,transfer,NA,Renewed,2019-10-13,25,1.18,ncyc,1,23,⋯,0.817,0.6676114,0.543,0.7268116,0.5644593,0.7075837,EU,0.817,0.6088277,0.817,0.6676114,0.543,0.7268116,0.5644593,0.7075837,18.33,"(15,25]",0.7075837,0.5644593,0.7268116,0.543,0.6676114,0.817,0.6088277,0.817
3,1&1 Internet,Germany,43576888,FirstTime,1,2020-01-31,2017-01-31,Active,audev.online,1&1 Internet,NA,3,55,18.33,0,287997921,auto-renewal,realized,Renewed,2020-01-31,25,2.57,audev,1,43,⋯,0.814,0.6877335,0.536,0.7155560,0.5601541,0.7314602,EU,0.814,0.6477227,0.814,0.6877335,0.536,0.7155560,0.5601541,0.7314602,18.33,"(15,25]",0.7314602,0.5601541,0.7155560,0.536,0.6877335,0.814,0.6477227,0.814
4,1&1 Internet,Germany,43727452,FirstTime,1,2020-02-03,2017-02-03,Active,pinkpanther.store,1&1 Internet,NA,3,88,29.33,0,288786056,auto-renewal,unrealized,Not Renewd,2020-02-03,40,7.27,pinkpanther,1,3,⋯,0.838,0.2588170,0.513,0.2754756,0.1642908,0.1535971,EU,0.833,0.2384536,0.838,0.2588170,0.513,0.2754756,0.1642908,0.1535971,29.33,"(25,35]",0.1535971,0.1642908,0.2754756,0.513,0.2588170,0.838,0.2384536,0.833
5,1&1 Internet,Germany,46911148,FirstTime,1,2020-04-17,2017-04-17,Active,lmengineeringservices.online,1&1 Internet,NA,3,50,16.67,0,308749164,auto-renewal,realized,Renewed,2020-04-17,25,7.81,lmengineeringservices,1,69,⋯,0.775,0.3878439,0.468,0.3962124,0.4979086,0.3769304,EU,0.772,0.3972606,0.775,0.3878439,0.468,0.3962124,0.4979086,0.3769304,16.67,"(15,25]",0.3769304,0.4979086,0.3962124,0.468,0.3878439,0.775,0.3972606,0.772
6,1&1 Internet,Germany,47605334,FirstTime,1,2020-05-08,2017-05-08,Active,eagleeyeview.store,1&1 Internet,NA,3,90,30.00,0,313867134,auto-renewal,unrealized,Not Renewd,2020-05-08,40,3.41,eagleeyeview,1,4,⋯,0.829,0.1730035,0.516,0.1675698,0.1723292,0.1027711,EU,0.823,0.1925156,0.829,0.1730035,0.516,0.1675698,0.1723292,0.1027711,30.00,"(25,35]",0.1027711,0.1723292,0.1675698,0.516,0.1730035,0.829,0.1925156,0.823


# Generate Metrics @ Tld-Reseller level 

In [10]:
l10_dplyr <- function (pred_df,
                              pred_var = "first_renewal_prediction") {
  N <- 10  # total number of rows to preallocate--possibly an overestimate
  lift_df <- data.frame(P =rep(NA, N), 
                        actu_renwd2=rep(NA, N), 
                        gain=rep(NA, N), 
                        lift=rep(NA, N), 
                        stringsAsFactors=FALSE)          # you don't know levels yet
  actu_renwd <- sum(pred_df[["renewal_status"]]=='Renewed')
  
  i = 1
  for(P in seq(.1,1,length=10)){
    temp_df <- data.frame(pred_df)[c("renewal_status",pred_var)]
    ttmp_df <- temp_df[order(temp_df[pred_var],decreasing = TRUE),][1:round(dim(temp_df)[1]*P),]
    actu_renwd2 <-  sum(ttmp_df[["renewal_status"]] == 'Renewed')
    gain = actu_renwd2/actu_renwd
    lift = gain/(P)
    
    lift_df[i, ] <- list(P, actu_renwd2, gain, lift)
    i = i+1
  }
  return(lift_df %>% filter(P==0.1) %>% pull(lift))
}

auc_dplyr <- function (pred_df,
                              pred_var = "first_renewal_prediction") {
  N <- 10  # total number of rows to preallocate--possibly an overestimate
  lift_df <- data.frame(P =rep(NA, N), 
                        actu_renwd2=rep(NA, N), 
                        gain=rep(NA, N), 
                        lift=rep(NA, N), 
                        stringsAsFactors=FALSE)          # you don't know levels yet
  actu_renwd <- sum(pred_df[["renewal_status"]]=='Renewed')
  
  i = 1
  for(P in seq(.1,1,length=10)){
    temp_df <- data.frame(pred_df)[c("renewal_status",pred_var)]
    ttmp_df <- temp_df[order(temp_df[pred_var],decreasing = TRUE),][1:round(dim(temp_df)[1]*P),]
    actu_renwd2 <-  sum(ttmp_df[["renewal_status"]] == 'Renewed')
    gain = actu_renwd2/actu_renwd
    lift = gain/(P)
    
    lift_df[i, ] <- list(P, actu_renwd2, gain, lift)
    i = i+1
  }
  return(calc_auc(lift_df))
}

In [11]:
preds_df %>% select(contains('pred_')) %>% names()

[1] "pred_agg_glm_ALL"      "pred_agg_rf_ALL"       "pred_agg_glm"         
 [4] "pred_agg_rf"           "pred_seg_glm_ALL"      "pred_seg_rf_ALL"      
 [7] "pred_seg2_glm_ALL"     "pred_seg2_rf_ALL"      "pred_agg_glm_ALL_fb"  
[10] "pred_agg_rf_ALL_fb"    "pred_agg_glm_fb"       "pred_agg_rf_fb"       
[13] "pred_seg_glm_ALL_fb"   "pred_seg_rf_ALL_fb"    "pred_seg2_glm_ALL_fb" 
[16] "pred_seg2_rf_ALL_fb"   "pred_seg2_rf_ALL_fb2"  "pred_seg2_glm_ALL_fb2"
[19] "pred_seg_rf_ALL_fb2"   "pred_seg_glm_ALL_fb2"  "pred_agg_rf_fb2"      
[22] "pred_agg_glm_fb2"      "pred_agg_rf_ALL_fb2"   "pred_agg_glm_ALL_fb2"

In [12]:
metrics_df <- preds_df %>%
  group_by(tld_registrar_index) %>%
  do( l10_seg2_glm = l10_dplyr(., pred_var = "pred_seg2_glm_ALL"),
      l10_seg_glm = l10_dplyr(., pred_var = "pred_seg_glm_ALL"),
       l10_agg_glm_ALL = l10_dplyr(., pred_var = "pred_agg_glm_ALL"),
       l10_agg_glm = l10_dplyr(., pred_var = "pred_agg_glm"),
       l10_seg2_rf = l10_dplyr(., pred_var = "pred_seg2_rf_ALL"),
       l10_seg_rf = l10_dplyr(., pred_var = "pred_seg_rf_ALL"),
       l10_agg_rf = l10_dplyr(., pred_var = "pred_agg_rf"),
       l10_agg_rf_ALL = l10_dplyr(., pred_var = "pred_agg_rf_ALL"),
     
      auc_seg2_glm = auc_dplyr(., pred_var = "pred_seg2_glm_ALL"),
      auc_seg_glm = auc_dplyr(., pred_var = "pred_seg_glm_ALL"),
       auc_agg_glm_ALL = auc_dplyr(., pred_var = "pred_agg_glm_ALL"),
       auc_agg_glm = auc_dplyr(., pred_var = "pred_agg_glm"),
       auc_seg2_rf = auc_dplyr(., pred_var = "pred_seg2_rf_ALL"),
       auc_seg_rf = auc_dplyr(., pred_var = "pred_seg_rf_ALL"),
       auc_agg_rf_ALL = auc_dplyr(., pred_var = "pred_agg_rf_ALL"),
       auc_agg_rf = auc_dplyr(., pred_var = "pred_agg_rf"),
      
     l10_seg2_glm_fb = l10_dplyr(., pred_var = "pred_seg2_glm_ALL_fb2"),
      l10_seg_glm_fb = l10_dplyr(., pred_var = "pred_seg_glm_ALL_fb2"),
       l10_agg_glm_fb = l10_dplyr(., pred_var = "pred_agg_glm_fb2"),
       l10_agg_glm_ALL_fb = l10_dplyr(., pred_var = "pred_agg_glm_ALL_fb2"),
       l10_seg2_rf_fb = l10_dplyr(., pred_var = "pred_seg2_rf_ALL_fb2"),
       l10_seg_rf_fb = l10_dplyr(., pred_var = "pred_seg_rf_ALL_fb2"),
       l10_agg_rf_fb = l10_dplyr(., pred_var = "pred_agg_rf_fb2"),
       l10_agg_rf_ALL_fb = l10_dplyr(., pred_var = "pred_agg_rf_ALL_fb2"),
     
      auc_seg2_glm_fb = auc_dplyr(., pred_var = "pred_seg2_glm_ALL_fb2"),
      auc_seg_glm_fb = auc_dplyr(., pred_var = "pred_seg_glm_ALL_fb2"),
       auc_agg_glm_fb = auc_dplyr(., pred_var = "pred_agg_glm_fb2"),
       auc_agg_glm_ALL_fb = auc_dplyr(., pred_var = "pred_agg_glm_ALL_fb2"),
       auc_seg2_rf_fb = auc_dplyr(., pred_var = "pred_seg2_rf_ALL_fb2"),
       auc_seg_rf_fb = auc_dplyr(., pred_var = "pred_seg_rf_ALL_fb2"),
       auc_agg_rf_fb = auc_dplyr(., pred_var = "pred_agg_rf_fb2"),
       auc_agg_rf_ALL_fb = auc_dplyr(., pred_var = "pred_agg_rf_ALL_fb2"),
      )

In [13]:
metrics_df2 <- metrics_df %>% mutate_if(is.list,as.numeric)

In [14]:
head(metrics_df2)

tld_registrar_index,l10_seg2_glm,l10_seg_glm,l10_agg_glm_ALL,l10_agg_glm,l10_seg2_rf,l10_seg_rf,l10_agg_rf,l10_agg_rf_ALL,auc_seg2_glm,auc_seg_glm,auc_agg_glm_ALL,auc_agg_glm,auc_seg2_rf,auc_seg_rf,auc_agg_rf_ALL,auc_agg_rf,l10_seg2_glm_fb,l10_seg_glm_fb,l10_agg_glm_fb,l10_agg_glm_ALL_fb,l10_seg2_rf_fb,l10_seg_rf_fb,l10_agg_rf_fb,l10_agg_rf_ALL_fb,auc_seg2_glm_fb,auc_seg_glm_fb,auc_agg_glm_fb,auc_agg_glm_ALL_fb,auc_seg2_rf_fb,auc_seg_rf_fb,auc_agg_rf_fb,auc_agg_rf_ALL_fb
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
fun1&1 internet,0.000000,0.000000,0.000000,0.000000,0.000000,1.666667,1.666667,1.666667,0.4666667,0.4500000,0.4500000,0.4500000,0.5333333,0.5333333,0.5500000,0.6166667,1.666667,1.666667,1.666667,1.666667,1.666667,1.666667,1.666667,1.666667,0.6500000,0.6500000,0.6500000,0.6500000,0.6500000,0.6500000,0.6500000,0.6500000
fun10dencehispahard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fun1api,0.000000,0.000000,2.500000,0.000000,0.000000,2.500000,0.000000,0.000000,0.4000000,0.4500000,0.5500000,0.4250000,0.3500000,0.4750000,0.3500000,0.5500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,0.4250000,0.4250000,0.4250000,0.4250000,0.4250000,0.4250000,0.4250000,0.4250000
funab name isp,3.333333,3.333333,3.333333,3.333333,3.333333,6.666667,6.666667,3.333333,0.5166667,0.7166667,0.5166667,0.4833333,0.6500000,0.6500000,0.6833333,0.7833333,6.666667,6.666667,6.666667,6.666667,6.666667,6.666667,6.666667,6.666667,0.8833333,0.8833333,0.8833333,0.8833333,0.8833333,0.8833333,0.8833333,0.8833333
funactive 24,2.000000,0.000000,2.000000,2.000000,4.000000,2.000000,0.000000,2.000000,0.6300000,0.3700000,0.4900000,0.4700000,0.6700000,0.6500000,0.6500000,0.6300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5300000,0.5300000,0.5300000,0.5300000,0.5300000,0.5300000,0.5300000,0.5300000
funalibaba,5.000000,5.028329,4.773371,4.730878,7.762040,7.691218,7.790368,7.832861,0.8390935,0.8284703,0.8087819,0.8077904,0.9164306,0.9165722,0.9198300,0.9189802,5.000000,5.028329,4.730878,4.773371,7.762040,7.691218,7.790368,7.832861,0.8390935,0.8284703,0.8077904,0.8087819,0.9164306,0.9165722,0.9189802,0.9198300


In [15]:
write.csv(metrics_df2, file="../../data/output/datapull_20201116/expiry_df_test_metrics.csv", row.names=FALSE)
system("gsutil cp /home/jupyter/local/Domains_202003/data/output/datapull_20201116/* gs://data_outputt/output/datapull_20201116/ ")
# system("mv /home/jupyter/local/Domains_202003/data/output/* /home/jupyter/local/Domains_202003/data/")

In [16]:
metrics_df <- metrics_df2

In [17]:
# metrics_df <- read.csv("../../data/../../data/output/datapull_20201116/expiry_df_test_metrics.csv")

In [18]:
# dim(metrics_df)
# head(metrics_df)

# Metalearning Prep

## add geo to training data for feature eng

In [19]:
# geo_suppl for train and test-pred data
geoLookupDF <- read.csv("/home/jupyter/Domains_202003/data/input/PredictiveModelAnalysis_ResellerGeoMap.csv")

In [20]:
preds_geo_df <- geo_suppl(preds_df, geoLookupDF = geoLookupDF)

Expiry data originally has 932347 rows and 932347 missing geo's.
... after intial merge on reseller & _country, expiry has 932347 rows and 8 missing geo's.
... after secondary fill with _country, expiry has 932347 rows and 8 missing geo's.
... after manual tweaks with _country, expiry has 932347 rows and 8 missing geo's.


In [21]:
# ensure same number of dims as before
dim(preds_df)
dim(preds_geo_df)

[1] 932347     59

[1] 932347     59

In [22]:
head(preds_geo_df)

,reseller,reseller_country,reseller_geo,domain_id,renewal_type,renewed_count,expiry_date,creation_date,status,domain,registrar,registrant_country,reg_period,reg_revenue,reg_arpt,renew_mbg,renewal_item_id,renew_type,autorenew_type,renewal_status,renew_date,renew_arpt,gibb_score,pattern,pattern_domain_count,⋯,pred_agg_rf_ALL,pred_agg_glm,pred_agg_rf,pred_seg_glm_ALL,pred_seg_rf_ALL,pred_seg2_glm_ALL,pred_seg2_rf_ALL,pred_agg_glm_ALL_fb,pred_agg_rf_ALL_fb,pred_agg_glm_fb,pred_agg_rf_fb,pred_seg_glm_ALL_fb,pred_seg_rf_ALL_fb,pred_seg2_glm_ALL_fb,pred_seg2_rf_ALL_fb,reg_arpt_org,reg_arpt_slab,pred_seg2_rf_ALL_fb2,pred_seg2_glm_ALL_fb2,pred_seg_rf_ALL_fb2,pred_seg_glm_ALL_fb2,pred_agg_rf_fb2,pred_agg_glm_fb2,pred_agg_rf_ALL_fb2,pred_agg_glm_ALL_fb2
,<fct>,<fct>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<dbl>,<dbl>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,,,NA,122049036,FirstTime,1,2020-08-30,2019-08-30,Active,qianxiyun.online,,NA,1,25,25.00,0,341703158,auto-renewal,unrealized,Not Renewd,2020-08-30,6,0.63,qianxiyun,1,⋯,0.9550391,NA,0.7713468,NA,NA,NA,NA,NA,0.9550391,NA,0.7713468,NA,NA,NA,NA,25.00,"(15,25]",0.5510000,0.5510000,0.5510000,0.551,0.7713468,0.551,0.9550391,0.551
2,1&1 Internet,Germany,EU,43450963,FirstTime,1,2020-01-27,2017-01-27,Active,ncyc.online,1&1 Internet,NA,3,55,18.33,0,239731665,transfer,NA,Renewed,2019-10-13,25,1.18,ncyc,1,⋯,0.6088277,0.817,0.6676114,0.543,0.7268116,0.5644593,0.7075837,0.817,0.6088277,0.817,0.6676114,0.543,0.7268116,0.5644593,0.7075837,18.33,"(15,25]",0.7075837,0.5644593,0.7268116,0.543,0.6676114,0.817,0.6088277,0.817
3,1&1 Internet,Germany,EU,43576888,FirstTime,1,2020-01-31,2017-01-31,Active,audev.online,1&1 Internet,NA,3,55,18.33,0,287997921,auto-renewal,realized,Renewed,2020-01-31,25,2.57,audev,1,⋯,0.6477227,0.814,0.6877335,0.536,0.7155560,0.5601541,0.7314602,0.814,0.6477227,0.814,0.6877335,0.536,0.7155560,0.5601541,0.7314602,18.33,"(15,25]",0.7314602,0.5601541,0.7155560,0.536,0.6877335,0.814,0.6477227,0.814
4,1&1 Internet,Germany,EU,43727452,FirstTime,1,2020-02-03,2017-02-03,Active,pinkpanther.store,1&1 Internet,NA,3,88,29.33,0,288786056,auto-renewal,unrealized,Not Renewd,2020-02-03,40,7.27,pinkpanther,1,⋯,0.2384536,0.838,0.2588170,0.513,0.2754756,0.1642908,0.1535971,0.833,0.2384536,0.838,0.2588170,0.513,0.2754756,0.1642908,0.1535971,29.33,"(25,35]",0.1535971,0.1642908,0.2754756,0.513,0.2588170,0.838,0.2384536,0.833
5,1&1 Internet,Germany,EU,46911148,FirstTime,1,2020-04-17,2017-04-17,Active,lmengineeringservices.online,1&1 Internet,NA,3,50,16.67,0,308749164,auto-renewal,realized,Renewed,2020-04-17,25,7.81,lmengineeringservices,1,⋯,0.3972606,0.775,0.3878439,0.468,0.3962124,0.4979086,0.3769304,0.772,0.3972606,0.775,0.3878439,0.468,0.3962124,0.4979086,0.3769304,16.67,"(15,25]",0.3769304,0.4979086,0.3962124,0.468,0.3878439,0.775,0.3972606,0.772
6,1&1 Internet,Germany,EU,47605334,FirstTime,1,2020-05-08,2017-05-08,Active,eagleeyeview.store,1&1 Internet,NA,3,90,30.00,0,313867134,auto-renewal,unrealized,Not Renewd,2020-05-08,40,3.41,eagleeyeview,1,⋯,0.1925156,0.829,0.1730035,0.516,0.1675698,0.1723292,0.1027711,0.823,0.1925156,0.829,0.1730035,0.516,0.1675698,0.1723292,0.1027711,30.00,"(25,35]",0.1027711,0.1723292,0.1675698,0.516,0.1730035,0.829,0.1925156,0.823


## Feature Engineering for tld-reseller level 

In [23]:
# new geo level feature eng
preds_meta_df = preds_geo_df %>%
  add_count(tld_registrar_index, reseller_geo) %>%
  group_by(tld_registrar_index) %>%
  summarise(
            geo_maj = reseller_geo[n == max(n)][1],
            geo_cnt = n_distinct(reseller_geo),
            n = n(),
            ren_prp = sum(renewal_status=='Renewed')/sum(n),
            tld_cnt = n_distinct(tld), tld_rat = tld_cnt/n,
              
            daydom_min = min(day_domains), 
            daydom_max = max(day_domains), 
            daydom_mean = mean(day_domains, na.rm = TRUE), 
            daydom_rng = daydom_max - daydom_min, 
            daydom_std = sd(day_domains, na.rm = TRUE), 
            daydom_skew = skewness(day_domains, na.rm = TRUE), 
            daydom_kurt = kurtosis(day_domains, na.rm = TRUE),

            sldlen_min = min(sld_length), 
            sldlen_max = max(sld_length), 
            sldlen_mean = mean(sld_length, na.rm = TRUE), 
            sldlen_rng = sldlen_max - sldlen_min, 
            sldlen_std = sd(sld_length, na.rm = TRUE), 
            sldlen_skew = skewness(sld_length, na.rm = TRUE), 
            sldlen_kurt = kurtosis(sld_length, na.rm = TRUE),
  
            gibbs_min = min(gibb_score), 
            gibbs_max = max(gibb_score), 
            gibbs_mean = mean(gibb_score, na.rm = TRUE), 
            gibbs_rng = gibbs_max - gibbs_min, 
            gibbs_std = sd(gibb_score, na.rm = TRUE), 
            gibbs_skew = skewness(gibb_score, na.rm = TRUE), 
            gibbs_kurt = kurtosis(gibb_score, na.rm = TRUE),
  
            pdcnt_min = min(pattern_domain_count), 
            pdcnt_max = max(pattern_domain_count), 
            pdcnt_mean = mean(pattern_domain_count, na.rm = TRUE), 
            pdcnt_rng = pdcnt_max - pdcnt_min, 
            pdcnt_std = sd(pattern_domain_count, na.rm = TRUE), 
            pdcnt_skew = skewness(pattern_domain_count, na.rm = TRUE), 
            pdcnt_kurt = kurtosis(pattern_domain_count, na.rm = TRUE),
  
            regarpt_min = min(reg_arpt), 
            regarpt_max = max(reg_arpt), 
            regarpt_mean = mean(reg_arpt, na.rm = TRUE), 
            regarpt_rng = regarpt_max - regarpt_min, 
            regarpt_std = sd(reg_arpt, na.rm = TRUE), 
            regarpt_skew = skewness(reg_arpt, na.rm = TRUE), 
            regarpt_kurt = kurtosis(reg_arpt, na.rm = TRUE))
dim(preds_meta_df)
head(preds_meta_df)

`summarise()` ungrouping output (override with `.groups` argument)



[1] 1758   42

tld_registrar_index,geo_maj,geo_cnt,n,ren_prp,tld_cnt,tld_rat,daydom_min,daydom_max,daydom_mean,daydom_rng,daydom_std,daydom_skew,daydom_kurt,sldlen_min,sldlen_max,sldlen_mean,sldlen_rng,sldlen_std,sldlen_skew,sldlen_kurt,gibbs_min,gibbs_max,gibbs_mean,gibbs_rng,gibbs_std,gibbs_skew,gibbs_kurt,pdcnt_min,pdcnt_max,pdcnt_mean,pdcnt_rng,pdcnt_std,pdcnt_skew,pdcnt_kurt,regarpt_min,regarpt_max,regarpt_mean,regarpt_rng,regarpt_std,regarpt_skew,regarpt_kurt
<fct>,<fct>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
fun1&1 internet,EU,1,11,0.54545455,1,9.090909e-02,1,5,2.181818,4,1.537412,0.9266651,-0.7811140,4,14,7.636364,10,2.907670,0.7720046,-0.3922768,1.71,9.01,4.655455,7.30,2.602016,6.453797e-01,-1.217221,1,3,1.363636,2,0.6741999,1.338593,0.3606612,15.0000,15.0,15.0000000,0.0000,0.0000000,NaN,NaN
fun10dencehispahard,EU,1,2,0.00000000,1,5.000000e-01,1,1,1.000000,0,0.000000,NaN,NaN,5,8,6.500000,3,2.121320,0.0000000,-2.7500000,3.28,8.62,5.950000,5.34,3.775950,9.898575e-17,-2.750000,1,1,1.000000,0,0.0000000,NaN,NaN,15.0000,15.0,15.0000000,0.0000,0.0000000,NaN,NaN
fun1api,EU,1,8,0.50000000,1,1.250000e-01,1,1,1.000000,0,0.000000,NaN,NaN,4,15,8.875000,11,3.522884,0.4848693,-1.1434523,2.00,6.09,4.396250,4.09,1.360167,-3.531195e-01,-1.301177,1,1,1.000000,0,0.0000000,NaN,NaN,3.0000,5.0,4.0000000,2.0000,1.0690450,0.000000,-2.234375
funab name isp,EU,1,47,0.06382979,1,2.127660e-02,1,119,82.872340,118,46.011632,-0.8738906,-0.9645903,4,61,9.446809,57,8.334863,5.0179710,28.2025571,0.28,17.92,4.695745,17.64,3.112366,1.526987e+00,4.826982,1,3,1.191489,2,0.4490714,2.194541,4.1889225,0.0001,15.0,0.8511511,14.9999,2.6207801,3.740087,15.752244
funactive 24,EU,1,25,0.20000000,1,4.000000e-02,1,14,5.040000,13,5.020624,1.0166170,-0.7554981,4,19,7.080000,15,3.451087,1.9087776,3.6548313,0.59,100.00,9.743600,99.41,19.035942,4.242864e+00,17.222688,1,10,1.520000,9,1.8055470,4.127909,16.4692321,0.5000,0.5,0.5000000,0.0000,0.0000000,NaN,NaN
funalibaba,China,1,35046,0.02014495,1,2.853393e-05,19,7163,1299.015380,7144,1600.896587,2.5691646,6.0230231,3,63,6.947155,60,3.018534,2.7839889,17.5227635,0.00,100.00,5.653614,100.00,19.616978,4.525337e+00,18.821500,1,400,23.168949,399,52.1502807,3.559110,14.2005513,0.0001,5.0,0.7338244,4.9999,0.1809528,3.717292,137.140359


In [24]:
# add a handful more vars 
country_maj = preds_df %>%
  add_count(tld_registrar_index, reseller_country) %>%
  group_by(tld_registrar_index) %>%
  mutate(reseller_country_maj = reseller_country[n == max(n)][1]) %>%
  select(-n) %>% 
  group_by(tld_registrar_index,reseller_country_maj) %>%
    summarise(n = n()) %>% 
    arrange(desc(n)) %>%
    pull(reseller_country_maj)

country_cnt = preds_df %>%
  add_count(tld_registrar_index, reseller_country) %>%
  group_by(tld_registrar_index) %>%
  summarise(reseller_country_cnt = n_distinct(reseller_country)) %>%
  pull(reseller_country_cnt)  

region_maj = preds_df %>%
  add_count(tld_registrar_index, region) %>%
  group_by(tld_registrar_index) %>%
  mutate(region_maj = region[n == max(n)][1]) %>%
  select(-n) %>% 
  group_by(tld_registrar_index,region_maj) %>%
    summarise(n = n()) %>% 
    arrange(desc(n)) %>%
    pull(region_maj)

region_cnt = preds_df %>%
  add_count(tld_registrar_index, region) %>%
  group_by(tld_registrar_index) %>%
  summarise(reseller_region_cnt = n_distinct(reseller_country)) %>%
  pull(reseller_region_cnt)  

preds_meta_df = preds_meta_df %>% 
    mutate(country_maj = country_maj,
            region_maj = region_maj,
            country_cnt = country_cnt,
            region_cnt = region_cnt,
            )

`summarise()` regrouping output by 'tld_registrar_index' (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` regrouping output by 'tld_registrar_index' (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)



In [25]:
dim(preds_meta_df)
head(preds_meta_df)

[1] 1758   46

tld_registrar_index,geo_maj,geo_cnt,n,ren_prp,tld_cnt,tld_rat,daydom_min,daydom_max,daydom_mean,daydom_rng,daydom_std,daydom_skew,daydom_kurt,sldlen_min,sldlen_max,sldlen_mean,sldlen_rng,sldlen_std,sldlen_skew,sldlen_kurt,gibbs_min,gibbs_max,gibbs_mean,gibbs_rng,gibbs_std,gibbs_skew,gibbs_kurt,pdcnt_min,pdcnt_max,pdcnt_mean,pdcnt_rng,pdcnt_std,pdcnt_skew,pdcnt_kurt,regarpt_min,regarpt_max,regarpt_mean,regarpt_rng,regarpt_std,regarpt_skew,regarpt_kurt,country_maj,region_maj,country_cnt,region_cnt
<fct>,<fct>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<int>,<int>
fun1&1 internet,EU,1,11,0.54545455,1,9.090909e-02,1,5,2.181818,4,1.537412,0.9266651,-0.7811140,4,14,7.636364,10,2.907670,0.7720046,-0.3922768,1.71,9.01,4.655455,7.30,2.602016,6.453797e-01,-1.217221,1,3,1.363636,2,0.6741999,1.338593,0.3606612,15.0000,15.0,15.0000000,0.0000,0.0000000,NaN,NaN,Japan,Non China,1,1
fun10dencehispahard,EU,1,2,0.00000000,1,5.000000e-01,1,1,1.000000,0,0.000000,NaN,NaN,5,8,6.500000,3,2.121320,0.0000000,-2.7500000,3.28,8.62,5.950000,5.34,3.775950,9.898575e-17,-2.750000,1,1,1.000000,0,0.0000000,NaN,NaN,15.0000,15.0,15.0000000,0.0000,0.0000000,NaN,NaN,United States,Non China,1,1
fun1api,EU,1,8,0.50000000,1,1.250000e-01,1,1,1.000000,0,0.000000,NaN,NaN,4,15,8.875000,11,3.522884,0.4848693,-1.1434523,2.00,6.09,4.396250,4.09,1.360167,-3.531195e-01,-1.301177,1,1,1.000000,0,0.0000000,NaN,NaN,3.0000,5.0,4.0000000,2.0000,1.0690450,0.000000,-2.234375,China,China,1,1
funab name isp,EU,1,47,0.06382979,1,2.127660e-02,1,119,82.872340,118,46.011632,-0.8738906,-0.9645903,4,61,9.446809,57,8.334863,5.0179710,28.2025571,0.28,17.92,4.695745,17.64,3.112366,1.526987e+00,4.826982,1,3,1.191489,2,0.4490714,2.194541,4.1889225,0.0001,15.0,0.8511511,14.9999,2.6207801,3.740087,15.752244,United States,Non China,1,1
funactive 24,EU,1,25,0.20000000,1,4.000000e-02,1,14,5.040000,13,5.020624,1.0166170,-0.7554981,4,19,7.080000,15,3.451087,1.9087776,3.6548313,0.59,100.00,9.743600,99.41,19.035942,4.242864e+00,17.222688,1,10,1.520000,9,1.8055470,4.127909,16.4692321,0.5000,0.5,0.5000000,0.0000,0.0000000,NaN,NaN,China,China,1,1
funalibaba,China,1,35046,0.02014495,1,2.853393e-05,19,7163,1299.015380,7144,1600.896587,2.5691646,6.0230231,3,63,6.947155,60,3.018534,2.7839889,17.5227635,0.00,100.00,5.653614,100.00,19.616978,4.525337e+00,18.821500,1,400,23.168949,399,52.1502807,3.559110,14.2005513,0.0001,5.0,0.7338244,4.9999,0.1809528,3.717292,137.140359,United States,Non China,1,1


# Join w/ Metrics

In [26]:
metametrics_df <- merge(preds_meta_df, metrics_df, on = 'tld_registrar_index', all = TRUE)

dim(metametrics_df)
head(metametrics_df)

[1] 1758   78

,tld_registrar_index,geo_maj,geo_cnt,n,ren_prp,tld_cnt,tld_rat,daydom_min,daydom_max,daydom_mean,daydom_rng,daydom_std,daydom_skew,daydom_kurt,sldlen_min,sldlen_max,sldlen_mean,sldlen_rng,sldlen_std,sldlen_skew,sldlen_kurt,gibbs_min,gibbs_max,gibbs_mean,gibbs_rng,⋯,l10_agg_rf_ALL,auc_seg2_glm,auc_seg_glm,auc_agg_glm_ALL,auc_agg_glm,auc_seg2_rf,auc_seg_rf,auc_agg_rf_ALL,auc_agg_rf,l10_seg2_glm_fb,l10_seg_glm_fb,l10_agg_glm_fb,l10_agg_glm_ALL_fb,l10_seg2_rf_fb,l10_seg_rf_fb,l10_agg_rf_fb,l10_agg_rf_ALL_fb,auc_seg2_glm_fb,auc_seg_glm_fb,auc_agg_glm_fb,auc_agg_glm_ALL_fb,auc_seg2_rf_fb,auc_seg_rf_fb,auc_agg_rf_fb,auc_agg_rf_ALL_fb
,<fct>,<fct>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,fun1&1 internet,EU,1,11,0.54545455,1,9.090909e-02,1,5,2.181818,4,1.537412,0.9266651,-0.7811140,4,14,7.636364,10,2.907670,0.7720046,-0.3922768,1.71,9.01,4.655455,7.30,⋯,1.666667,0.4666667,0.4500000,0.4500000,0.4500000,0.5333333,0.5333333,0.5500000,0.6166667,1.666667,1.666667,1.666667,1.666667,1.666667,1.666667,1.666667,1.666667,0.6500000,0.6500000,0.6500000,0.6500000,0.6500000,0.6500000,0.6500000,0.6500000
2,fun10dencehispahard,EU,1,2,0.00000000,1,5.000000e-01,1,1,1.000000,0,0.000000,NaN,NaN,5,8,6.500000,3,2.121320,0.0000000,-2.7500000,3.28,8.62,5.950000,5.34,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,fun1api,EU,1,8,0.50000000,1,1.250000e-01,1,1,1.000000,0,0.000000,NaN,NaN,4,15,8.875000,11,3.522884,0.4848693,-1.1434523,2.00,6.09,4.396250,4.09,⋯,0.000000,0.4000000,0.4500000,0.5500000,0.4250000,0.3500000,0.4750000,0.3500000,0.5500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,0.4250000,0.4250000,0.4250000,0.4250000,0.4250000,0.4250000,0.4250000,0.4250000
4,funab name isp,EU,1,47,0.06382979,1,2.127660e-02,1,119,82.872340,118,46.011632,-0.8738906,-0.9645903,4,61,9.446809,57,8.334863,5.0179710,28.2025571,0.28,17.92,4.695745,17.64,⋯,3.333333,0.5166667,0.7166667,0.5166667,0.4833333,0.6500000,0.6500000,0.6833333,0.7833333,6.666667,6.666667,6.666667,6.666667,6.666667,6.666667,6.666667,6.666667,0.8833333,0.8833333,0.8833333,0.8833333,0.8833333,0.8833333,0.8833333,0.8833333
5,funactive 24,EU,1,25,0.20000000,1,4.000000e-02,1,14,5.040000,13,5.020624,1.0166170,-0.7554981,4,19,7.080000,15,3.451087,1.9087776,3.6548313,0.59,100.00,9.743600,99.41,⋯,2.000000,0.6300000,0.3700000,0.4900000,0.4700000,0.6700000,0.6500000,0.6500000,0.6300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5300000,0.5300000,0.5300000,0.5300000,0.5300000,0.5300000,0.5300000,0.5300000
6,funalibaba,China,1,35046,0.02014495,1,2.853393e-05,19,7163,1299.015380,7144,1600.896587,2.5691646,6.0230231,3,63,6.947155,60,3.018534,2.7839889,17.5227635,0.00,100.00,5.653614,100.00,⋯,7.832861,0.8390935,0.8284703,0.8087819,0.8077904,0.9164306,0.9165722,0.9198300,0.9189802,5.000000,5.028329,4.730878,4.773371,7.762040,7.691218,7.790368,7.832861,0.8390935,0.8284703,0.8077904,0.8087819,0.9164306,0.9165722,0.9189802,0.9198300


# Write for model building


In [27]:
write.csv(metametrics_df, file="../../data/output/datapull_20201116/expiry_metametrics_df.csv", row.names=FALSE)
system("gsutil cp /home/jupyter/local/Domains_202003/data/output/datapull_20201116/* gs://data_outputt/output/datapull_20201116/ ")
